# Scraping for Question 3

Scraping necessary data to look at the most promising youth players and trying to predict what ratings they would achieve in future events.

Like previous research questions, we will be scraping the data from HLTV.org. Now we need to get every player rating from past events, as well as a list of important events to use for the prediction of young players careers later.

In [1]:
# Import libraries
import pandas as pd
import urllib
import urllib.request
import time
import re
import requests
from loguru import logger
from bs4 import BeautifulSoup

## Player Ratings in Past Events

Similarly to the first part of question 3, using the page of all the players ("https://www.hltv.org/stats/players"), I used BeatifulSoup to parse the HTML and a list of all the players pages is created by combining the scraped link and the baseURL

In [2]:
playersURLs = {"https://www.hltv.org/stats/players"}

baseURL = "https://www.hltv.org"

URL_List = list()
for page in playersURLs:
    # Logger message
    logger.info('Scraping new event page: %s.' % page)
    
    # Retrieve HTML
    req = urllib.request.Request(page, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    playerList = urllib.request.urlopen(req).read()
    
    # Sleep to prevent overflowing website
    time.sleep(2)
    
    # Parse HTML
    soup = BeautifulSoup(playerList, "html.parser")
    
    for item in soup.find_all(attrs={"class": "playerCol"}):
        for link in item.find_all("a"):
            URL_List.append(baseURL + link.get("href"))
  
print(URL_List)

2019-05-10 21:07:45.900 | INFO     | __main__:<module>:8 - Scraping new event page: https://www.hltv.org/stats/players.


['https://www.hltv.org/stats/players/11893/ZywOo', 'https://www.hltv.org/stats/players/7998/s1mple', 'https://www.hltv.org/stats/players/7938/XANTARES', 'https://www.hltv.org/stats/players/7131/BnTeT', 'https://www.hltv.org/stats/players/9216/coldzera', 'https://www.hltv.org/stats/players/9102/Sico', 'https://www.hltv.org/stats/players/15631/KSCERATO', 'https://www.hltv.org/stats/players/8950/Kaze', 'https://www.hltv.org/stats/players/10774/Freeman', 'https://www.hltv.org/stats/players/3741/NiKo', 'https://www.hltv.org/stats/players/8711/Sonic', 'https://www.hltv.org/stats/players/9960/frozen', 'https://www.hltv.org/stats/players/11271/smooya', 'https://www.hltv.org/stats/players/9172/XigN', 'https://www.hltv.org/stats/players/12553/yuurih', 'https://www.hltv.org/stats/players/7592/device', 'https://www.hltv.org/stats/players/9626/XELLOW', 'https://www.hltv.org/stats/players/5736/kNgV-', 'https://www.hltv.org/stats/players/8714/cosmeeeN', 'https://www.hltv.org/stats/players/12591/sterl

Having got the list of player URLs I placed the list into a dataframe and changed each of the URLs to the desired URLs and placed them back in a list.

In [3]:
df = pd.DataFrame({'col':URL_List})
df.col = [x.strip().replace('players/', 'players/events/') for x in df.col]
rating_List = df['col'].tolist()
print(rating_List)

['https://www.hltv.org/stats/players/events/11893/ZywOo', 'https://www.hltv.org/stats/players/events/7998/s1mple', 'https://www.hltv.org/stats/players/events/7938/XANTARES', 'https://www.hltv.org/stats/players/events/7131/BnTeT', 'https://www.hltv.org/stats/players/events/9216/coldzera', 'https://www.hltv.org/stats/players/events/9102/Sico', 'https://www.hltv.org/stats/players/events/15631/KSCERATO', 'https://www.hltv.org/stats/players/events/8950/Kaze', 'https://www.hltv.org/stats/players/events/10774/Freeman', 'https://www.hltv.org/stats/players/events/3741/NiKo', 'https://www.hltv.org/stats/players/events/8711/Sonic', 'https://www.hltv.org/stats/players/events/9960/frozen', 'https://www.hltv.org/stats/players/events/11271/smooya', 'https://www.hltv.org/stats/players/events/9172/XigN', 'https://www.hltv.org/stats/players/events/12553/yuurih', 'https://www.hltv.org/stats/players/events/7592/device', 'https://www.hltv.org/stats/players/events/9626/XELLOW', 'https://www.hltv.org/stats/p

Using this list of URLs, I scraped each page using BeatifulSoup, and filled 3 lists, the name of the player, the event he played in, and his rating in that event.

In [4]:
playername = list()
eventname = list()
eventrating = list()
for URL in rating_List:
    # Retrieve HTML
    req = urllib.request.Request(URL, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    playerList = urllib.request.urlopen(req).read()
    
    # Logger message
    logger.info('Scraping new players past ratings: %s' % URL)
    
    # Sleep to prevent overflowing website
    time.sleep(2)
    
    # Parse HTML
    soup = BeautifulSoup(playerList, "html.parser")
    
    #Name
    name_box = soup.find(attrs={"class": "context-item-name"})
    name = name_box.text.strip() # strip() is used to remove starting and trailing
    #Event and Rating
    for row in soup.findAll('table')[0].tbody.findAll('tr'):
        playername.append(name)
        eventname.append(row.findAll('td')[1].text)
        eventrating.append(row.findAll('td')[7].text)


2019-05-10 21:07:52.338 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/11893/ZywOo


2019-05-10 21:07:55.042 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/7998/s1mple


2019-05-10 21:07:58.291 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/7938/XANTARES


2019-05-10 21:08:01.559 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/7131/BnTeT


2019-05-10 21:08:04.882 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/9216/coldzera


...

2019-05-10 21:40:54.795 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/7217/SolEk


2019-05-10 21:40:57.681 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/3545/haste


2019-05-10 21:41:00.908 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/472/B1ad3


2019-05-10 21:41:04.851 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/7415/HUNDEN


2019-05-10 21:41:07.911 | INFO     | __main__:<module>:10 - Scraping new players past ratings: https://www.hltv.org/stats/players/events/6455/OCEAN


Having filled the lists, I placed them into a dictionary, and placed them into a Dataframe called dfRatings.

In [5]:
d = {'Player': playername, 'Event':eventname, 'Rating':eventrating}
dfRatings = pd.DataFrame(d)
dfRatings.sample(10)

,Player,Event,Rating
46822,rmn,WESG 2016 Europe & CIS Regional Finals,0.94
51834,vSa,ESEA MDL Season 26 North America,0.99
41964,SHOOWTiME,ECS Season 4 North America,0.97
46727,LOGAN,DreamHack Masters Dallas 2019 Europe Closed Qu...,0.94
35412,Relyks,CEVO Professional Season 8,0.99 *
39870,friberg,ESL Pro League Season 7 Europe,1.00
29736,ayaya,Douyu CS:GO Asia Invitational #2,1.11
47882,Coffee,ESL One Cologne 2017 - Europe Qualifier 2,0.75
45770,xms,ESG Tour Mykonos 2017 Closed Qualifier,1.14
55946,wayLander,FACEIT League 2015 Stage 2 EU Qualifier,0.75 *


I noticed that there was an * after any rating of less than one, so I stripped those from the datframe. 

In [6]:
dfRatings.Rating = [x.strip().replace(' *', '') for x in dfRatings.Rating]
dfRatings.sample(10)

,Player,Event,Rating
29687,MINISE,ESWC 2014 European Qualifier,0.84
26610,NENO,Operation: Kinguin #4,1.11
9792,k0nfig,Power-LAN 2016,1.25
45800,xms,Betway Invitational,1.18
43307,nbl,WCA 2017 South America Finals,1.04
16927,tabseN,ESWC 2014 European Qualifier,1.10
29519,MINISE,WePlay! Lock and Load,0.93
12645,DD,WCA 2017 China Qualifier #1,1.24
20794,ZOREE,Finnish Esports League Season 5 Finals,0.89
24470,kressy,ESEA MDL Season 29 Europe,0.95


## Important Events in Chronological Order

For the last part of scraping for this question, I scraped a list of important events (Events with a prizepool of over $4000).
I used BeatifulSoup to parse the webpage and collected all the event names from the page, and using the 'next page' link, I continued through all the pages that held the relevant event data.

In [7]:
# Functions

# Scrape Match URLs
def get_urls(soup, urlList):
    # Scrape match URLs from match results page
    event_box = soup.findAll(attrs={"class": "col-value event-col"})
    for e in event_box:
        urlList.append(e.text.strip())

    # Create URL for next page
    endURL = soup.find(attrs={"class": "pagination-next"}).get("href")
    return urlList, endURL

def get_html_soup(url):
    # Retrieve HTML using headers
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    page = urllib.request.urlopen(req).read()
    
    # Sleep to prevent overflowing website
    time.sleep(2)
    
    # Parse HTML
    soup = BeautifulSoup(page, "html.parser")
    return soup

def get_full_url(endURL):
    nextURL = "https://www.hltv.org" + endURL
    return nextURL

In [8]:
# Download example HTML to understand format
# Build opener to bypass 403 error
opener = urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3')]
urllib.request.install_opener(opener)

url = "https://www.hltv.org/events/archive?prizeMin=4000&prizeMax=1500000"
local = "EXAMPLE_HTML_2.html"

# Download example HTML file, save as example.html
urllib.request.urlretrieve(url,local)

# Sleep for 2 seconds
time.sleep(2)

In [9]:
# List for match URLS
eventURLList = list()

# Parse HTML of example HTML downloaded
soup = BeautifulSoup(open("EXAMPLE_HTML_2.html",encoding="utf8"), "html.parser")

# Scrape match URLs from match results page
eventURLList, endURL = get_urls(soup, eventURLList)

# Form URL
nextURL = get_full_url(endURL)

In [10]:
eventURLList

['Grosvenor CS:GO Championship Season 2',
 'WePlay! Forge of Masters Season 1 Finals',
 'IEM Sydney 2019',
 'The Elite Championship Series 2019',
 'LOOT.BET Season 2',
 'CSL 2019 Spring Finals',
 "Lotto Cup Winners' Cup",
 'T.O. Spawn',
 'ECS Season 7 North America Week 3',
 'ECS Season 7 Europe Week 3',
 'Hungarian National Esport Championship Season 1 Finals',
 'Insomnia64',
 'DreamHack Open Rio 2019',
 'CSadria Clan Championship Season 3 Finals',
 'Copenhagen Games 2019',
 'Copenhagen Games 2019 Female',
 'Asian Development League 2019 - Season 1',
 'ESEA Rank S Combine',
 'Charleroi Esports',
 'Baltic Esports League Season 2 Finals',
 'BLAST Pro Series Miami 2019',
 'LOOT.BET HotShot Series Season 2',
 'Cobx Masters',
 'StarSeries i-League Season 7',
 'ESEA Advanced Season 30 Europe',
 'ESL Meisterschaft Spring 2019 Finals',
 'LPL Pro League Season 1',
 'ESEA MDL Season 30 Europe',
 'ESEA Advanced Season 30 North America',
 'ESEA MDL Season 30 North America',
 'Iskandar Investment 

In [11]:
# While the next page returns a value, scrape
while(endURL != None):
    # Logger message
    #logger.info('Scraping new event page: %s' % nextURL)
    
    # Using urllib and BeautifulSoup, parse HTML page
    soup = get_html_soup(nextURL)
    
    # Scrape match URLs from match results page
    eventURLList, endURL = get_urls(soup, eventURLList)
    
    # Check to see if value returned is None, if not form next URL
    if endURL != None:
        nextURL = nextURL = get_full_url(endURL)

# Return list
eventURLList

['Grosvenor CS:GO Championship Season 2',
 'WePlay! Forge of Masters Season 1 Finals',
 'IEM Sydney 2019',
 'The Elite Championship Series 2019',
 'LOOT.BET Season 2',
 'CSL 2019 Spring Finals',
 "Lotto Cup Winners' Cup",
 'T.O. Spawn',
 'ECS Season 7 North America Week 3',
 'ECS Season 7 Europe Week 3',
 'Hungarian National Esport Championship Season 1 Finals',
 'Insomnia64',
 'DreamHack Open Rio 2019',
 'CSadria Clan Championship Season 3 Finals',
 'Copenhagen Games 2019',
 'Copenhagen Games 2019 Female',
 'Asian Development League 2019 - Season 1',
 'ESEA Rank S Combine',
 'Charleroi Esports',
 'Baltic Esports League Season 2 Finals',
 'BLAST Pro Series Miami 2019',
 'LOOT.BET HotShot Series Season 2',
 'Cobx Masters',
 'StarSeries i-League Season 7',
 'ESEA Advanced Season 30 Europe',
 'ESL Meisterschaft Spring 2019 Finals',
 'LPL Pro League Season 1',
 'ESEA MDL Season 30 Europe',
 'ESEA Advanced Season 30 North America',
 'ESEA MDL Season 30 North America',
 'Iskandar Investment 

Having scraped the important events, I placed them in a dataframe, and removed some extra characters. 

In [12]:
dfEvents = pd.DataFrame({'Event':eventURLList})
dfEvents.Event = [x.strip().replace('\n', '') for x in dfEvents.Event]
dfEvents

,Event
0,Grosvenor CS:GO Championship Season 2
1,WePlay! Forge of Masters Season 1 Finals
2,IEM Sydney 2019
3,The Elite Championship Series 2019
4,LOOT.BET Season 2
5,CSL 2019 Spring Finals
6,Lotto Cup Winners' Cup
7,T.O. Spawn
8,ECS Season 7 North America Week 3
9,ECS Season 7 Europe Week 3


I then saved both dataframes to CSV and saved them locally on my system. 

In [13]:
dfRatings.to_csv ('EVENT_RATINGS.csv', index = None, header=True)
dfEvents.to_csv ('PAST_EVENTS.csv', index = None, header=True)